In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mendeleev as md
import re

Now, the databasis are loaded

In [2]:
df = pd.read_csv('dfset_macro.csv')
db = pd.read_csv('qmof_formulas.csv')

Checking that compounds match in both databasis

In [ ]:
c = 0
for formula in df['formula'].values:
    if formula in db['formula'].values:
        c += 1

Column addition containing the formula in lexicographical fashion

In [4]:
db

,formula,name
0,Ba2 C6 Cu1 H14 O16,ABACUF01_FSR_macro
1,C36 Cu12 H56 I16 N4 S4,ABALOF_FSR_macro
2,C48 Co4 H32 N8 O16,ABAVIJ_FSR_macro
3,C48 Co4 H32 N8 O16,ABAVOP_FSR_macro
4,C50 H32 N6 O8 S4 Zn2,ABAXUZ_FSR_macro
...,...,...
16024,C88 H44 O20 Ti4,pyrene_NCVDL_macro
16025,C76 Cd4 H80 N12 S8,pyrene_NUHTIW_macro
16026,C176 Fe8 H96 O40,pyrene_PGHIM_macro
16027,C88 H54 O20 Zn4,pyrene_VUQLEA_macro


In [5]:
#diccio = {k:v for k,v in zip(db['cif'].values, db['lexform'].values)}
#df['lexform'] = [diccio.get(i,None) for i in df['cif']]
dat = pd.read_csv('datosrahm.csv')

symtoz = {k:v for k,v in zip(dat['Symbol'].values, dat['Z'].values)}
symtoz['D'] = 1

Creation of the dictionary to map element with the valence electron configuration

In [6]:
symtoconf = dict()
for el, z in zip(dat['Symbol'].values, dat['Z'].values):
    
    evc = [0, 0, 0, 0]
    z = int(z)
    
    if z <= 2 :
        vc = md.element(int(z)).econf.split()
    else:
        vc = md.element(int(z)).econf.split()[1:]
        
    vc = [i[1:] for i in vc]
    
    for item in vc:
        
        if len(item) ==1:
            item = item + '1'
        
        match = re.match(r"([a-z]+)([0-9]+)", item, re.I)
        items = match.groups()
        
        if items[0] == 's':
            evc[0] = int(items[1])/2
        if items[0] == 'p':
            evc[1] = int(items[1])/6
        if items[0] == 'd':
            evc[2] = int(items[1])/10
        if items[0] == 'f':
            evc[3] = int(items[1])/14
    symtoconf[el] =  evc

In [7]:
symtoconf['D'] = symtoconf['H']

The input data is built. The first dimension corresponds to the compound, the second one to the element in the formula, and the third one is the occupation fraction of the orbitals s, p, d, and f

In [9]:
atfrac = np.zeros((db.shape[0],96,4))
for row in range(db.shape[0]):
    formula = db['formula'][row]
    formula = formula.split()
    
    for item in formula:
        match = re.match(r"([a-z]+)([0-9]+)", item, re.I)
        items = match.groups()
        atfrac[row][symtoz.get(items[0],None)-1] = symtoconf.get(items[0],None)

In [10]:
(atfrac != 0).sum(axis=0)

array([[16029,     0,     0,     0],
       [    0,     0,     0,     0],
       [  260,     0,     0,     0],
       [    2,     0,     0,     0],
       [  113,   113,     0,     0],
       [16029, 16029,     0,     0],
       [13277, 13277,     0,     0],
       [13328, 13328,     0,     0],
       [  590,   590,     0,     0],
       [    0,     0,     0,     0],
       [  377,     0,     0,     0],
       [  130,     0,     0,     0],
       [   34,    34,     0,     0],
       [   56,    56,     0,     0],
       [  764,   764,     0,     0],
       [ 2702,  2702,     0,     0],
       [ 1600,  1600,     0,     0],
       [    0,     0,     0,     0],
       [  363,     0,     0,     0],
       [  237,     0,     0,     0],
       [   16,     0,    16,     0],
       [   16,     0,    16,     0],
       [  124,     0,   124,     0],
       [   31,     0,    31,     0],
       [  686,     0,   686,     0],
       [  266,     0,   266,     0],
       [  870,     0,   870,     0],
 

The input data with the valence electron configuration is saved, up to Z=96

In [11]:
np.save('element_features', atfrac[:,:96,:])

In [13]:
c = 0
for row in range(df.shape[0]):
    if db['name'][row] == df['name'][row]:
        c += 1

In [15]:
db['formula']

0            Ba2 C6 Cu1 H14 O16
1        C36 Cu12 H56 I16 N4 S4
2            C48 Co4 H32 N8 O16
3            C48 Co4 H32 N8 O16
4          C50 H32 N6 O8 S4 Zn2
                  ...          
16024           C88 H44 O20 Ti4
16025        C76 Cd4 H80 N12 S8
16026          C176 Fe8 H96 O40
16027           C88 H54 O20 Zn4
16028         C176 Ca8 H104 O40
Name: formula, Length: 16029, dtype: object